In [50]:
import requests as r

In [51]:
url = "http://localhost:8032"

In [52]:
r.get(url + "/healthz").json()

{'status': 'available',
 'system_information': {'environment': 'development', 'version': '0.0.1'}}

In [22]:
# wrong credentials
data = {"email": "dku@dku", "password": "Berlin"}
r.post(url + "/v1/tokens/authentication", json=data).json()

{'error': 'invalid authentication credentials'}

In [19]:
data = {"email": "dku@dku", "password": "berlin"}
res = r.post(url + "/v1/tokens/authentication", json=data).json()
res

{'authentication_token': {'token': 'ANXZ3APMOJ6SPIP3HXFN5ZMGPE',
  'expiry': '2023-03-11T23:31:35.348221+01:00'}}

In [20]:
tkn = res["authentication_token"]["token"]

In [45]:
tkn

'ANXZ3APMOJ6SPIP3HXFN5ZMGPE'

In [47]:
# correct token:
r.get(url + "/healthz", headers={"Authorization": f"Bearer {tkn}"}).json()

{'status': 'available',
 'system_information': {'environment': 'development', 'version': '0.0.1'}}

In [48]:
# wrong token:
r.get(url + "/healthz", headers={"Authorization": "incorrect"}).json()

{'error': 'invalid or missing authentication token'}

In [49]:
# anon get request (server treats as anonymous user):
r.get(url + "/healthz").json()

{'status': 'available',
 'system_information': {'environment': 'development', 'version': '0.0.1'}}

# Songs

In [85]:
r.get(url + "/songs").json()

[{'id': 'wish-you-were-here',
  'artist': 'Pink Floyd',
  'name': 'Wish You Were Here'},
 {'id': '', 'artist': 'Pink Floyd', 'name': 'Money'},
 {'id': 'money', 'artist': 'Pink Floyd', 'name': 'Money'},
 {'id': 'englishman-in-new-york',
  'artist': 'Sting',
  'name': 'Englishman In New York'},
 {'id': 'start-me-up', 'artist': 'The Rolling Stones', 'name': 'Start Me Up'}]

In [64]:
r.get(url + "/songs/start-me-up").json()

{'id': 'start-me-up',
 'artist': 'The Rolling Stones',
 'name': 'Start Me Up',
 'lyrics': "__Verse 1:__  \nIf you start me up!  \nIf you start me up I′ll never stop  \n\nIf you start me up!  \nIf you start me up I'll never stop  \n\nI′ve been running hot\nYou got me ticking gonna blow my top\n\nIf you start me up, if you start me up\nI'll never stop, never stop, never stop, never stop\n\nYou make a grown man cry\nYou make a grown man cry\nYou make a grown man cry\n\nSpread out the oil, the gasoline\nI walk smooth ride in a mean, mean machine\nStart it up!\n\n\n__Verse 2:__\n\nIf you start it up!\nKick on the starter, give it all you've got\n(You got, you got)\n\nI can′t compete\nWith the riders in the other heats\n\nIf you rough it up\nIf you like it you can slide it up, slide It up\nSlide it up, slide it up\n\nDon′t make a grown man cry\nDon't make a grown man cry\nDon′t make a grown man cry\n\nMy eyes dilate, my lips go green\nMy hands are greasy, she's a mean, mean machine\nStart it

### Without Authentication

In [65]:
# Create a song
data = {"artist": "Pink Floyd", "name": "Money"}
r.post(url + "/songs", json=data).json()

{'error': 'you must be authenticated to access this resource'}

In [90]:
# Delete a song
r.delete(url + "/songs/money", json=data).json()

{'error': 'you must be authenticated to access this resource'}

### With Authentication

In [83]:
# Create a song
data = {"artist": "Pink Floyd", "name": "Money", "id": "money"}
res1 = r.post(url + "/songs", json=data, headers={"Authorization": f"Bearer {tkn}"})

In [84]:
res1.text

'Success: Created New Song'

In [86]:
# Delete a song
res2 = r.delete(url + "/songs/money", json=data, headers={"Authorization": f"Bearer {tkn}"})

In [87]:
res2.text

'Success: Deleted Song with ID money'